In [1]:
import io
import os
import re
from xml.dom import minidom
import json
import codecs

Navigate to recipe directory:

In [2]:
#f = io.open("igt_cat.json", mode="r", encoding="utf-8")
#cwd = os.getcwd()
# Print the current working directory
#print("Current working directory: {0}".format(cwd))
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))
# f.read()
os.listdir(cwd)
res = []
# Iterate directory
for path in os.listdir(cwd):
    # check if current path is a file
    if os.path.isfile(os.path.join(cwd, path)):
        res.append(path)
print(len(res))

Current working directory: C:\Users\nlutt\Documents\Websites\ckCollections\recipes_xml\03 Mangold
877


Find out file name parts:

In [3]:
s = "recipe-1021541207378794 - Ananas - Erdbeer - Konfitüre.xml"
id = re.match(r"recipe-(\d+)", s).group(1)
print (id)

1021541207378794


In [4]:
rn = re.match(r"recipe-\d+\s-\s(.+)\.xml",s).group(1)
rn = re.sub(r"\s-\s", '-' ,rn)
print (rn)

Ananas-Erdbeer-Konfitüre


In [5]:
fn_new = rn + ' - r' + id + '.xml'
print (fn_new)

Ananas-Erdbeer-Konfitüre - r1021541207378794.xml


Find duplicate recipe names:

In [6]:
# Iterate directory
#for fn in os.listdir(cwd):
#    fn_new = re.match(r"(.*)\s-\sr", fn).group(1) + '.xml'
#    os.rename (fn, fn_new)
print (len(os.listdir(cwd)))

877


Read and parse ingredient catalogue:

In [7]:
with open('C:/Users/nlutt\Documents/Websites/Ingredients/igt-catalog.json', 'r', encoding='utf-8') as f:
  igtCat = json.load(f)
cat = igtCat.get('ingredients')
print (len(cat))
#for igt in cat:
#    print (igt.get('normName'))

720


In [8]:
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
fn = "Schalottenpfannekuchen mit Ziegenkäse gefüllt.xml"
recipe = minidom.parse (fn)
ingredients = recipe.getElementsByTagName("fr:igdtName")
rc = []
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
getIngredientNames (ingredients)
print (rc)

['Schalotte(n)', 'Olivenöl', 'Mehl', 'Milch', 'Ei(er)', 'Butter, flüssige', 'Mangold', 'Lauch', 'Käse (schnittfester Ziegenkäse)', 'Maiskörner, vorgegarte (Dose)', 'Knoblauch', 'Petersilie, fein geschnittene', 'Salz und Pfeffer aus der Mühle', 'Cayennepfeffer', 'Muskat, frisch gerieben']


Design middle and inner loop:

In [9]:
#middle loop: recipe ingredient name fields
for igt_in_rcp in rc:
    given = igt_in_rcp.lower()
    oder = re.match(r"(.*)\soder\s",given)
    if oder != None:
        given = oder.group(1)
    matches = []
    myMatch = {}
    #inner loop: normName and synonymes
    for igt in cat:
        x = igt.get('normName')
        m = re.search (x, given)
        if m != None:
            start, end = m.span()
            if end-start > 0:
                #print (end-start, '  ', m.group() )
                matching = {}
                matching['name']   = igt.get('id')
                matching['pos']    = start
                matching['length'] = end - start
                matches.append(matching) 
        synos = igt.get('synonyms')
        if synos != None:
            for syno in synos:
                m = re.search (syno, given)
                if m != None:
                    start, end = m.span()
                    if end-start > 0:
                        #print (end-start, '  ', m.group() )
                        matching = {}
                        matching['name']   = igt.get('id')
                        matching['pos']    = start
                        matching['length'] = end - start
                        matches.append(matching)
    if len(matches) > 0:
        matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
        myMatch = matches_sorted[0]
        print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
    if myMatch == {}:
        no_match_ctr += 1
#        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
        print (message)

     +  schalotte(n)                                 schalotten
     +  olivenöl                                     olivenöl
     +  mehl                                         weizenmehl
     +  milch                                        milch
     +  ei(er)                                       ei
     +  butter, flüssige                             butter
     +  mangold                                      mangold
     +  lauch                                        lauch
     +  käse (schnittfester ziegenkäse)              ziegenkäse
     +  maiskörner, vorgegarte (dose)                mais
     +  knoblauch                                    knoblauch
     +  petersilie, fein geschnittene                petersilie
     +  salz und pfeffer aus der mühle               
     +  cayennepfeffer                               cayennepfeffer
     +  muskat, frisch gerieben                      muskat


Design outer loop:

In [ ]:
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
                
i = 0
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
for fn in os.listdir(cwd):
    #print (fn)
    recipe = minidom.parse (fn)
    ingredients = recipe.getElementsByTagName("fr:igdtName")
    rc = []
    getIngredientNames (ingredients)
    i += 1
    #middle loop: recipe ingredient name fields
    for igt_in_rcp in rc:
        given = igt_in_rcp.lower()
        oder = re.match(r"(.*)\soder\s",given)
        if oder != None:
            given = oder.group(1)
        matches = []
        myMatch = {}
        #inner loop: normName and synonymes
        for igt in cat:
            x = igt.get('normName')
            m = re.search (x, given)
            if m != None:
                start, end = m.span()
                if end-start > 0:
                    #print (end-start, '  ', m.group() )
                    matching = {}
                    matching['name']   = igt.get('id')
                    matching['pos']    = start
                    matching['length'] = end - start
                    matches.append(matching) 
            synos = igt.get('synonyms')
            if synos != None:
                for syno in synos:
                    m = re.search (syno, given)
                    if m != None:
                        start, end = m.span()
                        if end-start > 0:
                            #print (end-start, '  ', m.group() )
                            matching = {}
                            matching['name']   = igt.get('id')
                            matching['pos']    = start
                            matching['length'] = end - start
                            matches.append(matching)
        if len(matches) > 0:
            matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
            myMatch = matches_sorted[0]
    #        print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
        if myMatch == {}:
            no_match_ctr += 1
    #        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
            message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
    #        print (message)

Adas bil Hamod.xml
Albertos Mangold-Ravioli mit Chickenwings und Shiitake-Pilze.xml
Albertos Mangold.xml
Alex vegane Mangold-Kokosmilch-Suppe.xml
Ananas-Erdbeer-Konfitüre.xml
Anjas Käse-Mangold-Rinderrouladen.xml
Apfel-Mangold-Tarte.xml
Arabische Sommersuppe.xml
Arap köftesi.xml
Artischocken-Mangold-Dip.xml
Asiatische Mangold-Nudeln mit Sesam.xml
Asiatisches Mangold-Fisch Gratin.xml
Asiatisches Schaschlik mit Mangold-Chutney.xml
Aufgespießte Steakrollen.xml
Auflauf mit Kartoffeln, Zucchini und Mangold.xml
Augsburger Grießnockerlsuppe mit Gemüse.xml
Avocado-Grapefruit Salat mit Paprikamandeln.xml
Ayurvedische Kartoffel-Mangold-Zucchini-Pfanne.xml
B&B Smoothie.xml
Babymangoldsalat.xml
Bambelsupp.xml
Bandnudeln mit Mangold-Ingwer-Soße.xml
Bandnudeln mit Mangold-Sahne-Sauce.xml
Bauchlappensteak-Räder.xml
Beefmakers Maultaschen.xml
Bergkäse-Kugeln mit Mangold.xml
Bitter Peach Symphonie.xml
Blattgemüse mit Garnelen.xml
Blattsalat mit Fisch und würzigem Gemüse.xml
Blitva.xml
Blätterteig-Quich

### Exchange recipe name
New and old recipe name:

In [12]:
fn_noext = fn[0:fn.index(".xml")]
print (fn_noext) 
rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
print (rn.nodeValue)

Alex vegane Mangold-Kokosmilch-Suppe
recipe-2876671439968626 - Alex vegane Mangold-Kokosmilch-Suppe


Replace:

In [13]:
rn.replaceWholeText(fn_noext)
print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)

Alex vegane Mangold-Kokosmilch-Suppe


Write to recipe file:

In [29]:
rcp_str = str(recipe.toxml())
with codecs.open('xx.xml', 'w', encoding = 'utf8') as file:
  file.write(rcp_str)
    